<a href="https://colab.research.google.com/github/LeoDinga/DL_Project/blob/main/Project_DL_3_Diana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --filter=blob:none --no-checkout https://github.com/THETIS-dataset/dataset.git
%cd dataset
!git sparse-checkout init --cone
!git sparse-checkout set VIDEO_Skelet3D
!git checkout

fatal: destination path 'dataset' already exists and is not an empty directory.
/content/dataset
Your branch is up to date with 'origin/main'.


In [8]:
%pip install --upgrade pip
%pip install mediapipe opencv-python
%pip install torch torchvision scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 143.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 155.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 152.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-

In [2]:
%pip install --force-reinstall --no-cache-dir mediapipe

import os
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
#import torchvision.transforms as T
#import torchvision.models as models
from PIL import Image
import numpy as np
#from sklearn.metrics import accuracy_score
import shutil
import random
import mediapipe as mp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 182.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 282.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 476.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 MB 267.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 172.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 271.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 276.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 359.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 409.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 446.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 290.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 431.9 MB/s eta 0:00:00
  Attempting uninstall: sentencepiece
    Found existi

/usr/local/lib/python3.11/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.5.1 is installed, but it is not compatible with the installed jaxlib version 0.6.0, so it will not be used.
  warnings.warn(


In [ ]:
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: False
Device name: No GPU


In [ ]:
import os
print(os.listdir('VIDEO_Skelet3D'))

['smash', 'forehand_openstands', 'kick_service', 'slice_service', 'backhand', 'backhand_slice', 'flat_service', 'backhand_volley', 'forehand_volley', 'forehand_slice', 'forehand_flat', 'backhand2hands']


In [ ]:
#shows the videos in each folder
data_path = 'VIDEO_Skelet3D'

# Print out the directory structure
for root, dirs, files in os.walk(data_path):
    print(f"Root: {root}")
    print(f"Dirs: {dirs}")
    print(f"Files: {files}")
    print("-" * 40)

Root: VIDEO_Skelet3D
Dirs: ['smash', 'forehand_openstands', 'kick_service', 'slice_service', 'backhand', 'backhand_slice', 'flat_service', 'backhand_volley', 'forehand_volley', 'forehand_slice', 'forehand_flat', 'backhand2hands']
Files: []
----------------------------------------
Root: VIDEO_Skelet3D/smash
Dirs: []
Files: ['p33_smash_skelet3D_s2.avi', 'p50_smash_skelet3D_s2.avi', 'p13_smash_skelet3D_s2.avi', 'p38_smash_skelet3D_s2.avi', 'p20_smash_skelet3D_s3.avi', 'p34_smash_skelet3D_s3.avi', 'p51_smash_skelet3D_s2.avi', 'p16_smash_skelet3D_s1.avi', 'p35_smash_skelet3D_s3.avi', 'p42_smash_skelet3D_s3.avi', 'p3_smash_skelet3D_s3.avi', 'p42_smash_skelet3D_s2.avi', 'p5_smash_skelet3D_s2.avi', 'p25_smash_skelet3D_s1.avi', 'p27_smash_skelet3D_s3.avi', 'p19_smash_skelet3D_s3.avi', 'p23_smash_skelet3D_s3.avi', 'p50_smash_skelet3D_s3.avi', 'p54_smash_skelet3D_s2.avi', 'p23_smash_skelet3D_s2.avi', 'p16_smash_skelet3D_s2.avi', 'p36_smash_skelet3D_s3.avi', 'p17_smash_skelet3D_s2.avi', 'p42_smash

In [ ]:
def convert_video_to_npy(video_path, resize_shape=(224, 224)):
    """
    Converts a video to a numpy array of resized frames.

    Parameters:
    - video_path: The path to the video file.
    - resize_shape: The desired frame size (default is 224x224).

    Returns:
    - frames_array: Numpy array containing all frames.
    """
    cap = cv2.VideoCapture(video_path)
    frames = []

    if not cap.isOpened():
        raise ValueError(f"Error opening video file: {video_path}")

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Resize frame to the desired shape
        frame_resized = cv2.resize(frame, resize_shape)
        frames.append(frame_resized)

    cap.release()

    # Convert list of frames into a numpy array
    frames_array = np.array(frames)
    return frames_array

def create_subfolders(pastas_dir, output_dir):
    """
    Creates subfolders in the output directory and processes .avi files.

    Parameters:
    - pastas_dir: The root directory containing class folders.
    - output_dir: The directory where the .npy files will be saved.
    """
    for pastas_filename in os.listdir(pastas_dir):
        origem_path = os.path.join(pastas_dir, pastas_filename)

        # Check if it's a directory (ignore files)
        if os.path.isdir(origem_path):
            nova_pasta_path = os.path.join(output_dir, pastas_filename)
            os.makedirs(nova_pasta_path, exist_ok=True)

            # Process each video file in the subfolder
            videos_dir = os.path.join(pastas_dir, pastas_filename)
            for video_filename in os.listdir(videos_dir):
                if video_filename.endswith(".avi"):  # Process only .avi files
                    video_path = os.path.join(videos_dir, video_filename)
                    video_name = os.path.splitext(video_filename)[0]

                    try:
                        # Convert the video to numpy array
                        frames_array = convert_video_to_npy(video_path)

                        # Save the numpy array to a .npy file
                        output_filename = os.path.join(nova_pasta_path, f"{video_name}.npy")
                        np.save(output_filename, frames_array)
                    except ValueError as e:
                        print(f"Error processing {video_filename}: {e}")
                    except Exception as e:
                        print(f"Unexpected error with {video_filename}: {e}")

# Example usage
pastas_dir = 'VIDEO_Skelet3D'  # Directory with .avi videos
npy_dir = 'npy_videos'  # Directory to save .npy files
os.makedirs(npy_dir, exist_ok=True)

# Process videos and maintain class folders
create_subfolders(pastas_dir, npy_dir)

In [ ]:
# Pad/truncate to 120 frames
def pad_or_truncate_keypoints(keypoints, target_length=120):
    """
    Pads or truncates a keypoints array to a fixed number of frames (target_length).
    If padding is needed, uses zeros.
    """
    current_length = keypoints.shape[0]

    if current_length == target_length:
        return keypoints
    elif current_length > target_length:
        return keypoints[:target_length]  # Truncate
    else:
        # Pad with zeros
        padding = np.zeros((target_length - current_length, keypoints.shape[1], keypoints.shape[2]))
        return np.concatenate([keypoints, padding], axis=0)

In [ ]:
def extract_keypoints(frames_array):
    keypoints = []
    for frame in frames_array:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)
        if results.pose_landmarks:
            keypoints.append([[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark])
        else:
            keypoints.append(np.zeros((33, 3)).tolist())
    keypoints = np.array(keypoints)
    keypoints = pad_or_truncate_keypoints(keypoints, target_length=120)
    return keypoints

In [ ]:
def split_by_player(video_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    # Step 1: Get a list of all players
    players = set()
    for video_filename in os.listdir(video_dir):
        if video_filename.endswith(".npy"):
            player_name = video_filename.split('_')[0]
            players.add(player_name)
    players = list(players)
    random.shuffle(players)

    # Step 2: Split players into train/val/test sets
    num_players = len(players)
    train_size = int(train_ratio * num_players)
    val_size = int(val_ratio * num_players)

    train_players = players[:train_size]
    val_players = players[train_size:train_size + val_size]
    test_players = players[train_size + val_size:]

    # Step 3: Create directories for train/val/test sets and keypoints
    train_dir = os.path.join(output_dir, 'train')
    val_dir = os.path.join(output_dir, 'val')
    test_dir = os.path.join(output_dir, 'test')

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Create keypoints subfolders
    for split_dir in [train_dir, val_dir, test_dir]:
        keypoints_dir = os.path.join(split_dir, 'keypoints')
        os.makedirs(keypoints_dir, exist_ok=True)

    # Step 4: Move videos and save keypoints
    for video_filename in os.listdir(video_dir):
        if video_filename.endswith(".npy"):
            player_name = video_filename.split('_')[0]
            src_path = os.path.join(video_dir, video_filename)

            if player_name in train_players:
                dst_dir = train_dir
            elif player_name in val_players:
                dst_dir = val_dir
            else:
                dst_dir = test_dir

            # Copy the video to the appropriate directory
            dst_video_path = os.path.join(dst_dir, video_filename)
            shutil.copy(src_path, dst_video_path)

            # Load the video and extract keypoints
            frames_array = np.load(src_path)
            keypoints = extract_keypoints(frames_array)

            # Save keypoints to the keypoints subfolder
            keypoints_filename = os.path.splitext(video_filename)[0] + '.npy'
            keypoints_path = os.path.join(dst_dir, 'keypoints', keypoints_filename)
            np.save(keypoints_path, keypoints)

In [ ]:
npy_dir = 'npy_videos'  # Pasta com os .npy dos vídeos
output_dir = 'split_players'  # Pasta de saída (serão criadas train, val, test)
os.makedirs(output_dir, exist_ok=True)

split_by_player(npy_dir, output_dir)

In [ ]:
# In theory this will extract the keypoints AND pad/truncate
from tqdm import tqdm

root_dir = "npy_videos"

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)

all_keypoints = {}

# Top-level loop with a single progress bar
action_folders = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
for action_folder in tqdm(action_folders, desc="Processing actions"):
    action_path = os.path.join(root_dir, action_folder)
    all_keypoints[action_folder] = {}

    video_files = sorted([f for f in os.listdir(action_path) if f.endswith(".npy")])
    for video_file in video_files:  # No tqdm here
        video_path = os.path.join(action_path, video_file)
        try:
            sample = np.load(video_path)
            video_keypoints = []

            for frame in sample:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = pose.process(frame_rgb)

                if results.pose_landmarks:
                    keypoints = [[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark]
                else:
                    keypoints = np.zeros((33, 3)).tolist()

                video_keypoints.append(keypoints)

            video_keypoints = np.array(video_keypoints)
            video_keypoints = pad_or_truncate_keypoints(video_keypoints, target_length=120)
            all_keypoints[action_folder][video_file] = video_keypoints

        except Exception as e:
            pass  # Suppress per-video error printing for a clean log

# Final summary
print("\n Summary of extracted keypoints:")
for action, videos in all_keypoints.items():
    print(f"- {action}: {len(videos)} videos processed.")

Processing actions:  25%|██▌       | 3/12 [10:01<29:42, 198.03s/it]